In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Etude du lien entre les différentes bases de données : une approche économétrique

On cherche à étudier le lien entre les différentes séries dont on dispose et le nombre officiel de touristes. Pour cela, on commence par s'assurer que les données à comparer soient comparables.

# Base du trafic aérien - nombre d'avions et de sièges par jour

On commence par reconstituer la base imputée du trafic aérien.(on effectue les mêmes commandes qu'auparavant).

In [2]:
df_air_traffic_france = pd.read_csv('france_air_traffic_210712.csv')
df_air_traffic_france['sheduled_time_start']=pd.to_datetime(df_air_traffic_france['sheduled_time_start'])
df_air_traffic_imput = df_air_traffic_france.copy()
df_imput_31dec = df_air_traffic_france[df_air_traffic_france["sheduled_time_start"]=='2018-12-30']
df_imput_31dec["Year"]=2018
df_imput_31dec["Month"]=12
df_imput_31dec["Day"]=31
df_imput_31dec["sheduled_time_start"] = pd.to_datetime(df_imput_31dec[['Year', 'Month', 'Day']])
df_imput_31dec['Year-Month'] = df_imput_31dec['sheduled_time_start'].dt.strftime('%Y-%m')
df_air_traffic_imput = pd.concat([df_air_traffic_imput, df_imput_31dec])
df_air_traffic_imput.sort_values(by=['sheduled_time_start'], inplace=True)
df_air_traffic_imput2 = df_air_traffic_imput.copy()
pb_2017 = pd.date_range('2017-05-16', '2017-12-31')
index_a_supprimer_2017 = df_air_traffic_imput2[df_air_traffic_imput2["sheduled_time_start"].isin(pb_2017)].index.tolist()
df_air_traffic_imput2.drop(index=index_a_supprimer_2017, axis=0, inplace=True)
dates_imput_2017 = pd.date_range('2018-05-16', '2018-12-31')
df_imput_obs_2017 = df_air_traffic_imput2[df_air_traffic_imput2["sheduled_time_start"].isin(dates_imput_2017)]
df_imput_obs_2017["Year"]=2017
df_imput_obs_2017["sheduled_time_start"] = pd.to_datetime(df_imput_obs_2017[['Year', 'Month', 'Day']])
df_imput_obs_2017['Year-Month'] = df_imput_obs_2017['sheduled_time_start'].dt.strftime('%Y-%m')
df_air_traffic_imput2 = pd.concat([df_air_traffic_imput2, df_imput_obs_2017])
df_air_traffic_imput2.sort_values(by=['sheduled_time_start'], inplace=True)

C:\Users\Clemence\AppData\Local\Temp/ipykernel_69840/21372328.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_imput_31dec["Year"]=2018
C:\Users\Clemence\AppData\Local\Temp/ipykernel_69840/21372328.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_imput_31dec["Month"]=12
C:\Users\Clemence\AppData\Local\Temp/ipykernel_69840/21372328.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cave

In [3]:
df_air_traffic_imput3 = df_air_traffic_imput2.groupby('sheduled_time_start', as_index=False).sum()

Pour chaque jour, on observe le nombre de sièges et d'avions. On supprime du df les colonnes inutiles.

In [4]:
df_air_traffic_imput3=df_air_traffic_imput3[["sheduled_time_start", "seats", "number_planes"]]

Comme les données officielles recensent le nombre de touristes arrivés en France par mois, le but est d'obtenir un nombre de vols et de sièges par mois.Pour cela, je sélectionne l'année et le mois pour chaque ligne. Ensuite, je crée une nouvelle variable Year-Month (compatible avec le format des données officielles) pour pouvoir faire la somme du nb d'avions et de sièges par mois et ensuite faire une jointure cohérente avec le df des données officielles.

In [5]:
df_air_traffic_imput3['Year'] = df_air_traffic_imput3['sheduled_time_start'].dt.year 
df_air_traffic_imput3['Month'] = df_air_traffic_imput3['sheduled_time_start'].dt.month 
df_air_traffic_imput3['Year-Month'] = df_air_traffic_imput3['sheduled_time_start'].dt.strftime('%Y-%m-01')

In [6]:
df_air_traffic_imput3

,sheduled_time_start,seats,number_planes,Year,Month,Year-Month
0,2017-12-31,150953.0,864,2017,12,2017-12-01
1,2018-01-01,110454.0,635,2018,1,2018-01-01
2,2018-01-02,111713.0,648,2018,1,2018-01-01
3,2018-01-03,113140.0,648,2018,1,2018-01-01
4,2018-01-04,114154.0,661,2018,1,2018-01-01
...,...,...,...,...,...,...
1433,2021-12-03,77261.0,442,2021,12,2021-12-01
1434,2021-12-04,62891.0,335,2021,12,2021-12-01
1435,2021-12-05,78047.0,437,2021,12,2021-12-01
1436,2021-12-06,61266.0,355,2021,12,2021-12-01


In [7]:
df_air_traffic_imput4 = df_air_traffic_imput3.groupby('Year-Month', as_index=False).sum()

In [8]:
df_air_traffic_imput4=df_air_traffic_imput4[["Year-Month", "seats", "number_planes"]]

Ainsi, pour chaque mois, j'obtiens le nombre de sièges et le nombre de vols.

In [9]:
df_air_traffic_imput4

,Year-Month,seats,number_planes
0,2017-12-01,150953.0,864
1,2018-01-01,3467197.0,20498
2,2018-02-01,3407715.0,20251
3,2018-03-01,4105976.0,24307
4,2018-04-01,3876068.0,22804
5,2018-05-01,4214402.0,25008
6,2018-06-01,4244152.0,25250
7,2018-07-01,4883670.0,28912
8,2018-08-01,4964357.0,28874
9,2018-09-01,4814087.0,28821


In [10]:
# je transforme la variable year-month en variable temps
df_air_traffic_imput4['Year-Month']=pd.to_datetime(df_air_traffic_imput4['Year-Month'], errors='coerce')

# Données officielles - nb de commentaires par mois

In [11]:
df_France_Official_Data = pd.read_csv('France_Official_Data.csv')
df_France_Official_Data['date']=pd.to_datetime(df_France_Official_Data['date'])

In [12]:
df_France_Official_Data['value_in_thousands']=pd.to_numeric(df_France_Official_Data['value_in_thousands'], errors='coerce')
df_France_Official_Data

,date,variable,department,individus,value_in_thousands
0,2011-01-01,Arrivées dans l'hôtellerie,Ain,Non-résidents,7.41
1,2011-02-01,Arrivées dans l'hôtellerie,Ain,Non-résidents,9.54
2,2011-03-01,Arrivées dans l'hôtellerie,Ain,Non-résidents,12.55
3,2011-04-01,Arrivées dans l'hôtellerie,Ain,Non-résidents,13.03
4,2011-05-01,Arrivées dans l'hôtellerie,Ain,Non-résidents,12.97
...,...,...,...,...,...
24955,2021-06-01,Arrivées dans l'hôtellerie,Yvelines,Résidents,65.91
24956,2021-07-01,Arrivées dans l'hôtellerie,Yvelines,Résidents,71.18
24957,2021-08-01,Arrivées dans l'hôtellerie,Yvelines,Résidents,56.66
24958,2021-09-01,Arrivées dans l'hôtellerie,Yvelines,Résidents,73.90


In [93]:
df_France_Official_Data[df_France_Official_Data["date"]=="2020-01-01"]

,date,variable,department,individus,value_in_thousands
108,2020-01-01,Arrivées dans l'hôtellerie,Ain,Non-résidents,NaN
238,2020-01-01,Arrivées dans l'hôtellerie,Ain,Résidents,NaN
368,2020-01-01,Arrivées dans l'hôtellerie,Aisne,Non-résidents,NaN
498,2020-01-01,Arrivées dans l'hôtellerie,Aisne,Résidents,NaN
628,2020-01-01,Arrivées dans l'hôtellerie,Allier,Non-résidents,NaN
...,...,...,...,...,...
24418,2020-01-01,Arrivées dans l'hôtellerie,Vosges,Résidents,NaN
24548,2020-01-01,Arrivées dans l'hôtellerie,Yonne,Non-résidents,NaN
24678,2020-01-01,Arrivées dans l'hôtellerie,Yonne,Résidents,NaN
24808,2020-01-01,Arrivées dans l'hôtellerie,Yvelines,Non-résidents,NaN


In [122]:
df_France_Official_Data2 = df_France_Official_Data.groupby('date', as_index=False).sum() 

#df_France_Official_Data2 = df_France_Official_Data.groupby('date', as_index=False).sum(min_count=1) 
#si on veut laisser les NaN et pas qu'ils soient remplacés par 0

Ici on a que les données par mois: chaque mois, on observe un nombre total de touristes sur le sol français.

In [123]:
df_France_Official_Data2[df_France_Official_Data2["date"]>"2018-01-01"]

,date,value_in_thousands
85,2018-02-01,7443.10
86,2018-03-01,9230.84
87,2018-04-01,9618.15
88,2018-05-01,10734.97
89,2018-06-01,11803.88
90,2018-07-01,12938.61
91,2018-08-01,13230.29
92,2018-09-01,11500.99
93,2018-10-01,10298.17
94,2018-11-01,8161.55


# Nombre de commentaires - nb par mois

In [15]:
df_reviews_us_france_210712 = pd.read_csv('reviews_us_france_210712.csv')
df_reviews_france=df_reviews_us_france_210712[df_reviews_us_france_210712['country']=="France"]
df_reviews_france['stay_date']=pd.to_datetime(df_reviews_france['stay_date'])
df_reviews_france['publication_date']=pd.to_datetime(df_reviews_france['publication_date'])

C:\Users\Clemence\AppData\Local\Temp/ipykernel_69840/2720934094.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_reviews_france['stay_date']=pd.to_datetime(df_reviews_france['stay_date'])
C:\Users\Clemence\AppData\Local\Temp/ipykernel_69840/2720934094.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_reviews_france['publication_date']=pd.to_datetime(df_reviews_france['publication_date'])


In [16]:
df_reviews_france2 = df_reviews_france.groupby("stay_date", as_index=False).sum()

ici c'est pareil, on a une observation par mois pour le nombre de commentaires laissés en considérant la date du séjour.

In [17]:
df_reviews_france2

,stay_date,number_of_reviewers
0,2018-07-01,936
1,2018-08-01,4342
2,2018-09-01,3614
3,2018-10-01,3477
4,2018-11-01,3755
5,2018-12-01,3842
6,2019-01-01,3148
7,2019-02-01,2856
8,2019-03-01,3254
9,2019-04-01,3936


# Base google trends - observations par jour

In [18]:
df_gt=pd.read_csv("google_trends_fr_1.csv")
df_gt['date']=pd.to_datetime(df_gt['date'])

In [19]:
df_gt2 = df_gt.reset_index().pivot_table(values = "value", index = "date", columns = "variable")
df_gt2=df_gt2.reset_index()

Le même problème que précédemment se pose: les données sont ici journalières. Comme les données officielles sont mensuelles, il faut faire une moyenne mensuelle ici - les données représentent cette fois ci non un total comme le nombre de vols ou de sièges mais sont un indice. Faire une somme n'aurait alors pas de sens.

In [20]:
df_gt2['Year'] = df_gt2['date'].dt.year 
df_gt2['Month'] = df_gt2['date'].dt.month 
df_gt2['Year-Month'] = df_gt2['date'].dt.strftime('%Y-%m-01')

In [21]:
df_gt3 = df_gt2.groupby('Year-Month', as_index=False).mean()

In [22]:
df_gt3

variable,Year-Month,aeroport nice cote d azur theme_cat_voyages,aeroport paris_cat_voyages,cote d azur theme_cat_voyages,disneyland paris hotel_cat_voyages,disneyland paris theme_cat_voyages,disneyland paris_cat_voyages,french riviera_cat_voyages,louvre_cat_voyages,lyon gare_cat_voyages,...,nice airport_cat_voyages,nice hotel_cat_voyages,nice_cat_voyages,paris gare_cat_voyages,paris hotel_cat_voyages,paris restaurant_cat_voyages,paris_cat_voyages,promenade des anglais_cat_voyages,Year,Month
0,2017-01-01,11.271290,3.965806,15.064194,36.698065,33.338710,41.417419,1.131290,10.424194,25.796129,...,7.272581,12.160000,26.329032,29.070000,17.042258,26.485161,35.481935,0.245161,2017.0,1.0
1,2017-02-01,11.683571,5.925000,17.884286,26.016429,31.757143,37.755714,1.491429,5.551429,31.879286,...,6.950000,19.350000,32.760000,50.788929,16.955714,20.282143,33.900000,0.285714,2017.0,2.0
2,2017-03-01,16.118387,2.361290,21.907742,33.387097,30.822581,37.252258,3.165806,14.763871,29.874839,...,10.358710,15.893226,37.266452,28.394516,16.641935,18.933871,32.660000,0.322581,2017.0,3.0
3,2017-04-01,15.310000,5.823000,25.233333,21.675000,36.384000,35.860000,5.215000,12.657667,29.925000,...,7.700000,14.950000,39.190667,35.734333,16.040000,21.633333,37.740000,0.462000,2017.0,4.0
4,2017-05-01,15.510000,6.025806,25.379355,21.507097,29.989032,39.237097,3.229032,9.855484,31.423548,...,11.604839,15.420645,36.708065,37.948387,14.286774,19.225806,37.953548,0.677419,2017.0,5.0
5,2017-06-01,15.882333,5.983333,42.267000,18.640000,29.172000,32.781000,4.494000,9.636000,36.965333,...,14.529000,19.443333,45.520000,34.696667,13.610333,25.775333,39.621333,2.086333,2017.0,6.0
6,2017-07-01,21.826452,8.320000,47.550968,32.166129,37.334839,39.140645,3.210968,11.125161,38.800000,...,17.940323,28.564516,50.501613,41.249032,16.258065,21.269032,39.468387,4.398065,2017.0,7.0
7,2017-08-01,19.661935,6.712258,33.743226,35.361290,39.642581,45.735484,4.185806,13.349032,33.380000,...,14.534194,15.527097,47.174194,37.109032,15.903226,28.722581,36.362903,4.434839,2017.0,8.0
8,2017-09-01,20.966667,6.600000,17.652000,30.940000,35.040000,40.500000,2.324000,12.392000,33.550000,...,14.040000,16.648667,34.503000,37.264000,17.783333,28.800000,37.425000,0.917000,2017.0,9.0
9,2017-10-01,14.240000,5.858065,17.663871,32.260000,38.878710,44.652258,1.317742,13.458710,35.260000,...,10.472903,14.623548,32.556129,40.480968,19.438710,25.091935,35.724194,0.000000,2017.0,10.0


In [23]:
# je transforme la variable year-month en variable temps
df_gt3['Year-Month']=pd.to_datetime(df_gt3['Year-Month'], errors='coerce')

# Fusion des df

In [124]:
# Fusion du nb d'avions et du nb de sièges et le nb de touristes offciels
df_fusion_aerien=df_France_Official_Data2.merge(df_air_traffic_imput4, how='inner', left_on="date", right_on="Year-Month")
df_fusion_aerien

,date,value_in_thousands,Year-Month,seats,number_planes
0,2017-12-01,8162.78,2017-12-01,150953.0,864
1,2018-01-01,7056.71,2018-01-01,3467197.0,20498
2,2018-02-01,7443.10,2018-02-01,3407715.0,20251
3,2018-03-01,9230.84,2018-03-01,4105976.0,24307
4,2018-04-01,9618.15,2018-04-01,3876068.0,22804
5,2018-05-01,10734.97,2018-05-01,4214402.0,25008
6,2018-06-01,11803.88,2018-06-01,4244152.0,25250
7,2018-07-01,12938.61,2018-07-01,4883670.0,28912
8,2018-08-01,13230.29,2018-08-01,4964357.0,28874
9,2018-09-01,11500.99,2018-09-01,4814087.0,28821


In [125]:
# Fusion des commentaires laissés par les touristes et le nb de touristes offciels
df_fusion_commentaires=df_France_Official_Data2.merge(df_reviews_france2, how='inner', left_on="date", right_on="stay_date")
df_fusion_commentaires.head(5)

,date,value_in_thousands,stay_date,number_of_reviewers
0,2018-07-01,12938.61,2018-07-01,936
1,2018-08-01,13230.29,2018-08-01,4342
2,2018-09-01,11500.99,2018-09-01,3614
3,2018-10-01,10298.17,2018-10-01,3477
4,2018-11-01,8161.55,2018-11-01,3755


In [126]:
# Fusion des gt et le nb de touristes offciels
df_fusion_gt=df_France_Official_Data2.merge(df_gt3, how='inner', left_on="date", right_on="Year-Month")
df_fusion_gt

,date,value_in_thousands,Year-Month,aeroport nice cote d azur theme_cat_voyages,aeroport paris_cat_voyages,cote d azur theme_cat_voyages,disneyland paris hotel_cat_voyages,disneyland paris theme_cat_voyages,disneyland paris_cat_voyages,french riviera_cat_voyages,...,nice airport_cat_voyages,nice hotel_cat_voyages,nice_cat_voyages,paris gare_cat_voyages,paris hotel_cat_voyages,paris restaurant_cat_voyages,paris_cat_voyages,promenade des anglais_cat_voyages,Year,Month
0,2017-01-01,6887.63,2017-01-01,11.271290,3.965806,15.064194,36.698065,33.338710,41.417419,1.131290,...,7.272581,12.160000,26.329032,29.070000,17.042258,26.485161,35.481935,0.245161,2017.0,1.0
1,2017-02-01,7304.45,2017-02-01,11.683571,5.925000,17.884286,26.016429,31.757143,37.755714,1.491429,...,6.950000,19.350000,32.760000,50.788929,16.955714,20.282143,33.900000,0.285714,2017.0,2.0
2,2017-03-01,8564.83,2017-03-01,16.118387,2.361290,21.907742,33.387097,30.822581,37.252258,3.165806,...,10.358710,15.893226,37.266452,28.394516,16.641935,18.933871,32.660000,0.322581,2017.0,3.0
3,2017-04-01,10106.67,2017-04-01,15.310000,5.823000,25.233333,21.675000,36.384000,35.860000,5.215000,...,7.700000,14.950000,39.190667,35.734333,16.040000,21.633333,37.740000,0.462000,2017.0,4.0
4,2017-05-01,10382.97,2017-05-01,15.510000,6.025806,25.379355,21.507097,29.989032,39.237097,3.229032,...,11.604839,15.420645,36.708065,37.948387,14.286774,19.225806,37.953548,0.677419,2017.0,5.0
5,2017-06-01,11560.11,2017-06-01,15.882333,5.983333,42.267000,18.640000,29.172000,32.781000,4.494000,...,14.529000,19.443333,45.520000,34.696667,13.610333,25.775333,39.621333,2.086333,2017.0,6.0
6,2017-07-01,12972.80,2017-07-01,21.826452,8.320000,47.550968,32.166129,37.334839,39.140645,3.210968,...,17.940323,28.564516,50.501613,41.249032,16.258065,21.269032,39.468387,4.398065,2017.0,7.0
7,2017-08-01,12835.78,2017-08-01,19.661935,6.712258,33.743226,35.361290,39.642581,45.735484,4.185806,...,14.534194,15.527097,47.174194,37.109032,15.903226,28.722581,36.362903,4.434839,2017.0,8.0
8,2017-09-01,11343.99,2017-09-01,20.966667,6.600000,17.652000,30.940000,35.040000,40.500000,2.324000,...,14.040000,16.648667,34.503000,37.264000,17.783333,28.800000,37.425000,0.917000,2017.0,9.0
9,2017-10-01,10065.70,2017-10-01,14.240000,5.858065,17.663871,32.260000,38.878710,44.652258,1.317742,...,10.472903,14.623548,32.556129,40.480968,19.438710,25.091935,35.724194,0.000000,2017.0,10.0


# Test de régression

In [27]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

## En prenant chaque df de manière séparée

In [127]:
# Nb de vols
X = sm.add_constant(df_fusion_aerien["number_planes"])
results = sm.OLS(df_fusion_aerien['value_in_thousands'], X).fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:     value_in_thousands   R-squared:                       0.587
Model:                            OLS   Adj. R-squared:                  0.578
Method:                 Least Squares   F-statistic:                     64.07
Date:                Wed, 02 Mar 2022   Prob (F-statistic):           3.40e-10
Time:                        23:46:47   Log-Likelihood:                -441.81
No. Observations:                  47   AIC:                             887.6
Df Residuals:                      45   BIC:                             891.3
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const           816.8940    895.384      0.912

In [128]:
# Nb de sièges
X = sm.add_constant(df_fusion_aerien["seats"])
results = sm.OLS(df_fusion_aerien['value_in_thousands'], X).fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:     value_in_thousands   R-squared:                       0.573
Model:                            OLS   Adj. R-squared:                  0.564
Method:                 Least Squares   F-statistic:                     60.42
Date:                Wed, 02 Mar 2022   Prob (F-statistic):           7.39e-10
Time:                        23:46:55   Log-Likelihood:                -442.61
No. Observations:                  47   AIC:                             889.2
Df Residuals:                      45   BIC:                             892.9
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        748.7646    927.034      0.808      0.4

In [129]:
# Nb de commentaires
X = sm.add_constant(df_fusion_commentaires["number_of_reviewers"])
results = sm.OLS(df_fusion_commentaires['value_in_thousands'], X).fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:     value_in_thousands   R-squared:                       0.401
Model:                            OLS   Adj. R-squared:                  0.385
Method:                 Least Squares   F-statistic:                     25.39
Date:                Wed, 02 Mar 2022   Prob (F-statistic):           1.18e-05
Time:                        23:47:00   Log-Likelihood:                -385.58
No. Observations:                  40   AIC:                             775.2
Df Residuals:                      38   BIC:                             778.5
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
const                1479.9236   1

In [130]:
# Google trends
X = sm.add_constant(df_fusion_gt.drop(["date", "value_in_thousands", "Year-Month", "Year", "Month"], axis = 1))
results = sm.OLS(df_fusion_gt['value_in_thousands'], X).fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:     value_in_thousands   R-squared:                       0.873
Model:                            OLS   Adj. R-squared:                  0.767
Method:                 Least Squares   F-statistic:                     8.221
Date:                Wed, 02 Mar 2022   Prob (F-statistic):           6.25e-08
Time:                        23:47:08   Log-Likelihood:                -508.25
No. Observations:                  58   AIC:                             1070.
Df Residuals:                      31   BIC:                             1126.
Df Model:                          26                                         
Covariance Type:            nonrobust                                         
                                                  coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------

On met en log car les données des google trends c'est un indice donc ce sera plus représentatif d'interpréter en log.

In [134]:
columns = list(df_fusion_gt.columns.values)
columns.remove("date")
columns.remove("Year-Month")

df_fusion_gt_log = np.log(df_fusion_gt[columns]) #transforme toutes les variables en log
df_fusion_gt_log.replace([np.inf, -np.inf], np.nan, inplace=True) 
df_fusion_gt_log.dropna(inplace=True) 


X = sm.add_constant(df_fusion_gt_log.drop(["value_in_thousands", "Year", "Month"], axis = 1))
results = sm.OLS(df_fusion_gt_log['value_in_thousands'], X).fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:     value_in_thousands   R-squared:                       0.924
Model:                            OLS   Adj. R-squared:                  0.800
Method:                 Least Squares   F-statistic:                     7.448
Date:                Wed, 02 Mar 2022   Prob (F-statistic):           6.27e-05
Time:                        23:54:41   Log-Likelihood:                 54.697
No. Observations:                  43   AIC:                            -55.39
Df Residuals:                      16   BIC:                            -7.842
Df Model:                          26                                         
Covariance Type:            nonrobust                                         
                                                  coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------

# Régression sur l'ensemble des variables

In [33]:
df_fusion1=df_fusion_aerien.merge(df_fusion_commentaires, how='inner', on="date")
df_fusion1=df_fusion1[["date", "seats", "number_planes", "number_of_reviewers"]]

In [34]:
df_fusion2=df_fusion1.merge(df_fusion_gt, how='inner', on="date")

In [35]:
df_fusion2

,date,seats,number_planes,number_of_reviewers,value_in_thousands,Year-Month,aeroport nice cote d azur theme_cat_voyages,aeroport paris_cat_voyages,cote d azur theme_cat_voyages,disneyland paris hotel_cat_voyages,...,nice airport_cat_voyages,nice hotel_cat_voyages,nice_cat_voyages,paris gare_cat_voyages,paris hotel_cat_voyages,paris restaurant_cat_voyages,paris_cat_voyages,promenade des anglais_cat_voyages,Year,Month
0,2018-07-01,4883670.0,28912,936,12938.61,2018-07-01,21.666129,6.603548,46.451613,39.100000,...,20.823871,26.189032,52.373226,38.956129,18.385161,23.145161,41.803548,5.832258,2018.0,7.0
1,2018-08-01,4964357.0,28874,4342,13230.29,2018-08-01,25.010323,6.223871,43.318710,34.376452,...,17.723871,17.987097,52.316129,41.555806,17.450968,25.519355,38.888710,1.914839,2018.0,8.0
2,2018-09-01,4814087.0,28821,3614,11500.99,2018-09-01,16.146667,5.250000,20.408667,45.456667,...,13.216000,15.360000,39.366000,49.375000,22.200000,22.230000,40.016667,0.100000,2018.0,9.0
3,2018-10-01,4780669.0,28277,3477,10298.17,2018-10-01,17.264516,3.579677,9.570968,41.063226,...,9.967742,14.930323,38.388387,49.592903,17.566452,31.252903,38.183226,0.410968,2018.0,10.0
4,2018-11-01,3450107.0,20388,3755,8161.55,2018-11-01,13.640000,4.596000,8.499333,26.514667,...,4.810000,11.544000,26.268000,41.064667,16.233000,29.300000,37.238000,1.280000,2018.0,11.0
5,2018-12-01,2669261.0,15424,3842,7995.92,2018-12-01,12.696774,5.460968,11.145806,19.709677,...,5.229677,8.643871,29.546452,32.985161,12.141290,23.287742,33.234839,0.819677,2018.0,12.0
6,2019-01-01,3553036.0,20469,3148,7020.50,2019-01-01,13.276774,6.467742,19.700645,37.905484,...,5.675806,16.276129,35.288710,34.639355,18.919355,31.367742,40.443871,0.225806,2019.0,1.0
7,2019-02-01,3562193.0,20856,2856,7582.23,2019-02-01,12.955357,5.007143,17.619286,30.129643,...,6.908571,15.621429,38.202500,41.114643,15.093571,28.945000,38.674286,0.142857,2019.0,2.0
8,2019-03-01,3984509.0,23365,3254,8979.44,2019-03-01,8.491935,5.636129,22.129032,38.980000,...,12.043226,13.250000,35.096774,48.195484,16.854839,29.320968,38.935484,0.675161,2019.0,3.0
9,2019-04-01,4609542.0,27047,3936,10352.06,2019-04-01,22.464667,7.921333,34.568000,28.772333,...,14.160667,16.740000,43.725000,52.026333,17.273000,31.826667,26.748000,2.062667,2019.0,4.0


Ici, on fait la régression du nb total de touristes sur toutes nos variables: pas forcément pertinent. trop de variables?

In [36]:
# Régression sur la base finale
X = sm.add_constant(df_fusion2.drop(["date", "value_in_thousands", "Year", "Month", "Year-Month"], axis = 1))
results = sm.OLS(df_fusion2['value_in_thousands'], X).fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:     value_in_thousands   R-squared:                       0.974
Model:                            OLS   Adj. R-squared:                  0.900
Method:                 Least Squares   F-statistic:                     13.11
Date:                Wed, 02 Mar 2022   Prob (F-statistic):           8.28e-05
Time:                        21:23:47   Log-Likelihood:                -322.53
No. Observations:                  40   AIC:                             705.1
Df Residuals:                      10   BIC:                             755.7
Df Model:                          29                                         
Covariance Type:            nonrobust                                         
                                                  coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------

Ici on fait la régression du nb total de touristes sur toutes nos variables sans les GG trends pour voir les coeff significatifs :

In [44]:
df_fusion_sansGGtrend = df_fusion_aerien.merge(df_fusion_commentaires, how='inner', on="date")
df_fusion_sansGGtrend = df_fusion_sansGGtrend[["date", "value_in_thousands_x", "seats", "number_planes", "number_of_reviewers"]]
df_fusion_sansGGtrend.rename(columns={"value_in_thousands_x": "value_in_thousands"}, inplace=True)
df_fusion_sansGGtrend

,date,value_in_thousands,seats,number_planes,number_of_reviewers
0,2018-07-01,12938.61,4883670.0,28912,936
1,2018-08-01,13230.29,4964357.0,28874,4342
2,2018-09-01,11500.99,4814087.0,28821,3614
3,2018-10-01,10298.17,4780669.0,28277,3477
4,2018-11-01,8161.55,3450107.0,20388,3755
5,2018-12-01,7995.92,2669261.0,15424,3842
6,2019-01-01,7020.50,3553036.0,20469,3148
7,2019-02-01,7582.23,3562193.0,20856,2856
8,2019-03-01,8979.44,3984509.0,23365,3254
9,2019-04-01,10352.06,4609542.0,27047,3936


In [46]:
# Régression sur la base finale
X = sm.add_constant(df_fusion_sansGGtrend.drop(["date", "value_in_thousands"], axis = 1))
results = sm.OLS(df_fusion_sansGGtrend['value_in_thousands'], X).fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:     value_in_thousands   R-squared:                       0.794
Model:                            OLS   Adj. R-squared:                  0.777
Method:                 Least Squares   F-statistic:                     46.21
Date:                Wed, 02 Mar 2022   Prob (F-statistic):           1.98e-12
Time:                        21:39:12   Log-Likelihood:                -364.23
No. Observations:                  40   AIC:                             736.5
Df Residuals:                      36   BIC:                             743.2
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
const                -982.8560    